In [1]:
!nvidia-smi

Thu Jul  4 17:36:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:07:00.0 Off |                    0 |
| N/A   60C    P0            363W /  400W |   17218MiB /  81920MiB |    100%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
# Choose GPUs with enough memories
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
import pandas as pd

contem_2023 = pd.read_csv("/shared/share_tm-finance/Processed_df/One_year_window/contem_2023.csv")

In [4]:
contem_2023.head()

,date,rp_entity_id,comnam,ret,headline,vocab_con_headline
0,2023-01-03,3DC887,ARISTA NETWORKS INC,-0.003626,Network Monitoring Global Market Report 2022: ...,network monitoring sector reach 3.8 2030 cagr
1,2023-01-03,3DC887,ARISTA NETWORKS INC,-0.003626,Network Monitoring Global Market Report 2022: ...,network monitoring corporations continue onlin...
2,2023-01-03,3DC887,ARISTA NETWORKS INC,-0.003626,iRocket Appoints Kelyn Brannon to Board of Dir...,irocket appoints kelyn brannon board directors
3,2023-01-03,8EA478,MODERNA INC,-0.003507,Official List Official List Notice -2-,official list official list notice
4,2023-01-03,8EA478,MODERNA INC,-0.003507,DelveInsight Evaluates a Robust Cystic Fibrosi...,delveinsight evaluates robust cystic fibrosis ...


In [5]:
contem_2023.shape

(1749222, 6)

In [6]:
from sentence_transformers import SentenceTransformer
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

/user/wx2309/.conda/envs/TM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = embedding_model.encode(contem_2023.headline.tolist(),show_progress_bar=True)

In [ ]:
# import numpy as np

# os.mkdir("/user/wx2309/Topic-modeling-store/Bertopic_sp500_2023_contem_model_folder")
# with open("/user/wx2309/Topic-modeling-store/Bertopic_sp500_2023_contem_model_folder/contem_2023_embeddings.npy", "wb") as f:
#     np.save(f, embeddings)
# headline_list = contem_2023.headline.tolist()

In [7]:
import numpy as np
embeddings = np.load("/shared/share_tm-finance/Embeddings/One_year_window/contem_2023_embeddings.npy")
headline_list = contem_2023.vocab_con_headline.tolist()

In [8]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors = 50, n_components = 5, min_dist = 0, metric = 'cosine',random_state = 42,verbose = True)
# Upon several fittings, it was found that cluster size around 1000 to a 1.7M headlines create 50-100 topics
hdbscan_model = HDBSCAN(min_cluster_size =1000,  metric='euclidean', cluster_selection_method='eom',\
                        gen_min_span_tree=True,prediction_data=False,min_samples = 50,verbose = True)
vectorizer_model = CountVectorizer()
Topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model,
                       calculate_probabilities = False,verbose = True,low_memory = True)

In [9]:
Topic_model.fit(headline_list,embeddings)
Topic_model.get_topic_info()
# os.mkdir("/user/wx2309/Topic-modeling-store/Bertopic_sp500_2023_contem_model_folder/Bertopic_sp500_2023_contem_model")
# Topic_model.save("/user/wx2309/Topic-modeling-store/Bertopic_sp500_2023_contem_model_folder/Bertopic_sp500_2023_contem_model",serialization = "safetensors", save_ctfidf = True, save_embedding_model = embedding_model)

2024-07-04 17:44:15,594 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[D] [17:44:18.813601] /opt/conda/conda-bld/work/cpp/src/umap/runner.cuh:107 n_neighbors=50
[D] [17:44:18.814465] /opt/conda/conda-bld/work/cpp/src/umap/runner.cuh:129 Calling knn graph run
[D] [17:47:20.918168] /opt/conda/conda-bld/work/cpp/src/umap/runner.cuh:135 Done. Calling fuzzy simplicial set
[D] [17:47:20.997263] /opt/conda/conda-bld/work/cpp/src/umap/fuzzy_simpl_set/naive.cuh:318 Smooth kNN Distances
[D] [17:47:20.998941] /opt/conda/conda-bld/work/cpp/src/umap/fuzzy_simpl_set/naive.cuh:320 sigmas = [ 0.0884554, 0.109777, 0.154414, 6.59121e-05, 0.136098, 0.00510345, 0.000392279, 0.00346686, 0.000243207, 0.000329672, 0.00681055, 0.193758, 0.000290023, 0.000279798, 0.000243207, 0.15864, 7.25302e-05, 0.000205597, 0.15864, 0.000240855, 0.0831985, 0.00346714, 0.0906198, 0.000418326, 0.000240855 ]

[D] [17:47:20.998991] /opt/conda/conda-bld/work/cpp/src/umap/fuzzy_simpl_set/naive.cuh:322 rhos = [ 5.96046e-08, 4.76837e-07, 1.19209e-07, 1.19209e-07, 5.96046e-08, 0.00124151, 0.192946, 2.

2024-07-04 17:52:23,108 - BERTopic - Dimensionality - Completed ✓
2024-07-04 17:52:23,142 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-04 17:55:11,556 - BERTopic - Cluster - Completed ✓
2024-07-04 17:55:11,783 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-04 17:55:20,167 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,0,1747864,0_00_investor_size_morgan,"[00, investor, size, morgan, annual, rosen, se...","[atmos cut 117.00 125.00 morgan stanley, ambar..."
1,1,1358,1_calendar_expected_week_,"[calendar, expected, week, , , , , , , ]","[calendar expected week, calendar expected wee..."


In [10]:
# Topic_model = BERTopic.load("/user/wx2309/Topic-modeling-store/Bertopic_sp500_2023_contem_model_folder/Bertopic_sp500_2023_contem_model",
#                            embedding_model = embedding_model)

In [11]:
topic_dist, _ = Topic_model.approximate_distribution(headline_list)
contem_ret_topic_dist = pd.concat([contem_2023.drop(columns = ["rp_entity_id","headline","vocab_con_headline"]),pd.DataFrame(topic_dist)],axis = 1)
grouped = contem_ret_topic_dist.groupby(['date',"comnam","ret"])
grouped_sum = grouped.sum()

  0%|          | 6/1750 [00:00<01:07, 25.75it/s]

100%|██████████| 1750/1750 [01:07<00:00, 26.04it/s]


In [12]:
X = np.array(grouped_sum)
ret = []
for ind in list(grouped_sum.index):
  ret.append(ind[2])
Y = np.array(ret).reshape(-1,1)
from sklearn.linear_model import LinearRegression
regression = LinearRegression(fit_intercept=True)
regression.fit(X,Y)
regression.score(X,Y)

0.00015762267005081654